In [6]:
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

os.makedirs("plots", exist_ok=True)

# Load JSON data

with open("vfx_logs.json") as f:
    logs = json.load(f)

df = pd.json_normalize(logs)

# Clean and validate the data

def clean_vfx_logs(df):
    # Convert timestamp to datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    # Drop data with missing or invalid data
    df = df.dropna(subset=[
        'timestamp', 'user_id', 'department','task_type', 'status', 'duration_sec'])

    # Filter to known values
    valid_statuses = ['in_progress', 'completed', 'failed', 'queued']
    valid_departments = ['Lighting', 'Compositing', 'FX', 'Modeling','Rigging', 'Animation', 'Layout', 'Rendering']

    df = df[df['status'].isin(valid_statuses)]
    df = df[df['department'].isin(valid_departments)]

    # Check if duration is a number val
    df['duration_sec'] = pd.to_numeric(df['duration_sec'], errors='coerce')
    df = df.dropna(subset=['duration_sec'])

    return df

df_clean = clean_vfx_logs(df)

# Check to see tasks are within a given time and then categorize by length
df_clean['long_task'] = df_clean['duration_sec'] > 7200  # > 2 hours
df_clean['is_failed'] = df_clean['status'] == 'failed'

# Count long and failed tasks
long_task_count = df_clean['long_task'].sum()
failed_task_count = df_clean['is_failed'].sum()

print(f"🚨 Long tasks: {long_task_count}, Failed tasks: {failed_task_count}")

# Create new data from the raw data (can be used for later use)

df_clean['hour'] = df_clean['timestamp'].dt.hour
df_clean['day'] = df_clean['timestamp'].dt.date

# Summary table

summary = df_clean.groupby('department')['duration_sec'].agg(
    ['count', 'mean', 'median', 'max']
).reset_index()
summary.to_csv("plots/summary_by_department.csv", index=False)

user_summary = df_clean.groupby('user_id')['duration_sec'].agg(['count', 'mean', 'max']).sort_values(by='count', ascending=False)
user_summary.to_csv("plots/user_summary.csv")

df_clean.to_csv("plots/cleaned_vfx_logs.csv", index=False)
summary.to_csv("plots/department_summary.csv", index=False)

df_clean[df_clean['is_failed']].to_csv("plots/failed_tasks.csv", index=False)
df_clean[df_clean['long_task']].to_csv("plots/long_tasks.csv", index=False)


# Final data visualization to be used for reports and further analyses

sns.set(style="whitegrid")

# Plot 1:Task Duration by Department
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_clean, x='department', y='duration_sec')
plt.title("Task Duration by Department")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("plots/task_duration_by_department.png")
plt.close()

# Plot 2:Number of Tasks Over Time
plt.figure(figsize=(10, 6))
df_clean.groupby('day').size().plot(kind='line', marker='o')
plt.title("Tasks Logged Over Time")
plt.ylabel("Number of Tasks")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("plots/tasks_over_time.png")
plt.close()

# Plot 3:Hourly Activity Heatmap by Department
pivot = df_clean.pivot_table(index='hour', columns='department', values='log_id', aggfunc='count').fillna(0)
plt.figure(figsize=(12, 6))
sns.heatmap(pivot, cmap='YlGnBu', linewidths=.5, annot=True, fmt=".0f")
plt.title("Activity Heatmap: Hour vs Department")
plt.tight_layout()
plt.savefig("plots/activity_heatmap.png")
plt.close()


🚨 Long tasks: 0, Failed tasks: 236
